This notebook contains the code that was used to create csv files with assets price data.

In [3]:
import pandas as pd
import investiny

# Stocks

In [93]:
stocks = ['AAPL', 'AMD', 'TSLA', 'AMZN', 'NVDA', 'INTC', 'MU', 'MSFT', 'META', 'GOOGL']

In [94]:
EXCHANGE_STOCKS = 'NASDAQ'
investing_id_stocks = {}
for ticker in stocks:
    results = investiny.search_assets(query=ticker, type='Stock', exchange=EXCHANGE_STOCKS)
    for result in results:
        if result['full_name'] == f'{EXCHANGE_STOCKS}:{ticker}':
            break
    print(result)
    investing_id_stocks[ticker] = result['ticker']

{'symbol': 'AAPL', 'full_name': 'NASDAQ:AAPL', 'description': 'Apple Inc', 'type': 'Stock', 'ticker': '6408', 'exchange': 'NASDAQ'}
{'symbol': 'AMD', 'full_name': 'NASDAQ:AMD', 'description': 'Advanced Micro Devices Inc', 'type': 'Stock', 'ticker': '8274', 'exchange': 'NASDAQ'}
{'symbol': 'TSLA', 'full_name': 'NASDAQ:TSLA', 'description': 'Tesla Inc', 'type': 'Stock', 'ticker': '13994', 'exchange': 'NASDAQ'}
{'symbol': 'AMZN', 'full_name': 'NASDAQ:AMZN', 'description': 'Amazon.com Inc', 'type': 'Stock', 'ticker': '6435', 'exchange': 'NASDAQ'}
{'symbol': 'NVDA', 'full_name': 'NASDAQ:NVDA', 'description': 'NVIDIA Corporation', 'type': 'Stock', 'ticker': '6497', 'exchange': 'NASDAQ'}
{'symbol': 'INTC', 'full_name': 'NASDAQ:INTC', 'description': 'Intel Corporation', 'type': 'Stock', 'ticker': '251', 'exchange': 'NASDAQ'}
{'symbol': 'MU', 'full_name': 'NASDAQ:MU', 'description': 'Micron Technology Inc', 'type': 'Stock', 'ticker': '8092', 'exchange': 'NASDAQ'}
{'symbol': 'MSFT', 'full_name':

In [95]:
investing_id_stocks

{'AAPL': '6408',
 'AMD': '8274',
 'TSLA': '13994',
 'AMZN': '6435',
 'NVDA': '6497',
 'INTC': '251',
 'MU': '8092',
 'MSFT': '252',
 'META': '26490',
 'GOOGL': '6369'}

In [69]:
# For the following assets, the data source on investing.com is NASDAQ
investing_id_stocks = {
    'AAPL': 6408,
    'AMD': 8274,
    'TSLA': 13994,
    'AMZN': 6435,
    'NVDA': 6497,
    'INTC': 251,
    'MU': 8092,
    'MSFT': 252,
    'META': 26490,
    'GOOGL': 6369
}

# Commodities

For commodities, investing.com id's were searched manually to ensure the correct ticker is choosed.

In [ ]:
investiny.search_assets(query='ZC', type='Commodity')

In [108]:
# For the following assets, the data source on investing.com is capital.com
investing_id_commodities = {
    'Brent Oil Futures': 8833,
    'Crude Oil WTI Futures': 8849,
    'Natural Gas Futures': 8862,
    'Heating Oil Futures': 8988,
    'Gold Futures': 8830,
    'Silver Futures': 8836,
    'Copper Futures': 8831,  # ticker: HG, exchange: COMEX
    'Platinum Futures': 8910,
    'US Coffee C Futures': 8832,
    'US Corn Futures': 8918,
}

# Cryptocurrencies

For crypto, investing.com id's were also searched manually.

There was no USDT/USD data on Binance data source, so for this pair I used the data provided by Kraken.

Also, for DOGE/USD I used Huobi as data source, because Binance only had the data starting from 2022-09-14.

In [ ]:
investiny.search_assets(query='DOGE/USD', limit=100)

In [217]:
# For the following assets, the data source on investing.com is Binance,
# unless otherwise specified
investing_id_crypto = {
    'ETH': 1058142,
    'USDT': 1031397,  # exchange: Kraken
    'USDC': 1142432,
    'BNB': 1054919,
    'XRP': 1075586,
    'BUSD': 1172746,
    'ADA': 1073899,
    # 'SOL': 1173409,
    'DOGE': 1128789,  # exchange: Huobi
}

# Download data

In [218]:
def get_df(investing_id, from_date, to_date, actual_min_date) -> pd.DataFrame:
    """Creates pandas.DataFrame with historical market data for given financial asset.

    All parameters, except `actual_min_date`, are passed to investiny.historical_data().
    See documentation for this function for the meaning of these parameters.

    The date `from_date` should be at least a week before than `actual_min_date` to make
    sure that the data contains the previous trading day before from_date. This is needed
    because for some assets there is no data available on 01/01/2020 (the minimum possible
    value of parameter `from_date` in returns generating functions in `data.py`).

    IMPORTANT: all dates must be in format 'MM/DD/YYYY'

    :param actual_min_date: the minimum possible value of parameter `from_date`
        in returns generating functions in `data.py`
    :return: pandas.DataFrame with historical market data, including one previous
        trading day before the date `from_date`.
    """
    data_dict = investiny.historical_data(investing_id, from_date, to_date)
    df = pd.DataFrame.from_dict(data_dict)
    df['date'] = pd.to_datetime(df['date'])
    # index of the row that is previous to `actual_min_date`
    previous_to_min = df.loc[df['date'] == actual_min_date].index.item() - 1
    df = df.iloc[previous_to_min:]
    return df

In [219]:
# dates format: MM/DD/YYYY
actual_min_date = '01/02/2020'
from_date = '12/01/2019'
to_date = '10/01/2022'

In [71]:
for ticker in stocks:
    df = get_df(investing_id_stocks[ticker], from_date, to_date, actual_min_date)
    df.to_csv(f'data/stocks/{ticker}.csv', index=False)

In [112]:
for name in investing_id_commodities.keys():
    df = get_df(investing_id_commodities[name], from_date, to_date, actual_min_date)
    df.to_csv(f'data/commodities/{name}.csv', index=False)

In [220]:
for name in investing_id_crypto.keys():
    df = get_df(investing_id_crypto[name], from_date, to_date, actual_min_date)
    df.to_csv(f'data/crypto/{name}.csv', index=False)

For BTC, I could not find data from Binance on investing.com using `investiny`, so I manually downloaded the csv data from [https://www.investing.com/crypto/bitcoin/btc-usd?cid=1035793](https://www.investing.com/crypto/bitcoin/btc-usd?cid=1035793) and renamed the columns.

In [228]:
def process_raw_csv(in_path, out_path):
    """ Converts the csv with historical data from investing.com to more usable format """
    df = pd.read_csv(in_path)
    df = df.rename(columns={'Date': 'date', 'Price': 'close', 'Open': 'open', 'High': 'high', 'Low': 'low', 'Vol.': 'volume'})
    df = df.drop(columns=['Change %'])
    df = df[['date', 'open', 'high', 'low', 'close', 'volume']]

    df['date'] = pd.to_datetime(df['date'])
    for col_name in ['open', 'high', 'low', 'close']:
        df[col_name] = df[col_name].str.replace(',', '', regex=False).astype(float)
    df['volume'] = df['volume'].str.replace('K', '', regex=False).astype(float) * 1000
    df['volume'] = df['volume'].round().astype(int)
    df = df.iloc[::-1]
    df.to_csv(out_path, index=False)

In [229]:
process_raw_csv(in_path='data/crypto/BTC_raw.csv', out_path='data/crypto/BTC.csv')